[View in Colaboratory](https://colab.research.google.com/github/FranciscoRencoret1/deeplearning/blob/master/AlexNet.ipynb)

# Tarea 1 Deep Learning

Francisco Rencoret (FranciscoRencoret1) - Raimundo Manterola (rmant)

In [3]:
!wget https://www.dropbox.com/s/c0d7npcg1yysnux/MIT-10-Classes.zip?dl=0
!unzip -q MIT-10-Classes.zip?dl=0

--2018-04-21 19:34:28--  https://www.dropbox.com/s/c0d7npcg1yysnux/MIT-10-Classes.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/L5S6Z1kwskaXIroZmvk3ElWSovtMspDVicM7d1Rdc0zQAleCbOAFD96dtiofctEw/file [following]
--2018-04-21 19:34:29--  https://dl.dropboxusercontent.com/content_link/L5S6Z1kwskaXIroZmvk3ElWSovtMspDVicM7d1Rdc0zQAleCbOAFD96dtiofctEw/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.7.6, 2620:100:6016:6::a27d:106
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.7.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 139459806 (133M) [application/zip]
Saving to: ‘MIT-10-Classes.zip?dl=0’

MIT-10-Classes.zip? 100%[===================>] 133.00M  33.7MB/s    in 4.0s    


In [0]:
#Definicion de librerias con la funciones que seran utilizadas por Keras.
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical

In [6]:
#Definicion de contenedor y primera capa de AlexNet.
modelAlexNet = Sequential()
modelAlexNet.add(ZeroPadding2D((2,2), input_shape=(224, 224, 3)))
modelAlexNet.add(Convolution2D(96, (11,11), strides=(4,4), padding="valid"))
modelAlexNet.add(Activation(activation="relu"))
output_shape_first_layer = modelAlexNet.output_shape
print('Los mapas de activación de la capa 1 de convolción tienen tamaño: {}'.format(output_shape_first_layer))
modelAlexNet.add(BatchNormalization())
modelAlexNet.add(MaxPooling2D((3,3), strides=(2,2)))

Los mapas de activación de la capa 1 de convolción tienen tamaño: (None, 55, 55, 96)


In [7]:
#Definicion de la segunda capa de AlexNet.
modelAlexNet.add(ZeroPadding2D((2,2)))
modelAlexNet.add(Convolution2D(256, (5, 5), padding="valid"))
modelAlexNet.add(Activation(activation="relu"))
output_shape_second_layer = modelAlexNet.output_shape
print('Los mapas de activación de la capa 2 de convolción tienen tamaño: {}'.format(output_shape_second_layer))
modelAlexNet.add(BatchNormalization())
modelAlexNet.add(MaxPooling2D((3,3), strides=(2,2)))

Los mapas de activación de la capa 2 de convolción tienen tamaño: (None, 27, 27, 256)


Actividad 1

Notamos que la primera capa de AlexNet es de tamaño (55, 55, 96). Como podemos ver, nuestra primera capa tiene las mismas dimensiones:

In [99]:
output_shape_first_layer

(None, 55, 55, 96)

La segunda capa de AlexNet es de tamanño (27, 27, 256). Nuestra segunda capa tiene las mismas dimensiones

In [100]:
output_shape_second_layer

(None, 27, 27, 256)

Ahora, vamos a mostrar un resumen de la red creada

In [101]:
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
__________

Podemos notar que en la primera capa, teniamos filtros de tamaño 11 x 11 x 3 + 1 pesos (11 x 11 con profundidad 3 + el bias). Ahora, como la primera capa tiene 96 mapas de activacion, vamos a tener 96 de estos filtros, por lo que en la primera capa se tiene (11 x 11 x 3 + 1) x 96 = 34,944 pesos. Podemos ver que esto coincide con el valor de la segunda fila de la tabla.

La segunda capa tiene filtros de 5 x 5 x 96 + 1 (5 x 5 con profundidad 96 + un bias) y tenemos 256 mapas de activacion, por lo que en total la capa tiene (5 x 5 x 96 + 1) x 256 = 614,656. Esto coincide con el valor de la septima fila de la tabla.

Por otra parte, podemos notar que el batch normaliation en ambos casos calcula 4 parametros. Como la normalización consiste en:

$$\begin{equation}
\hat{a} =  \frac{x - \mu}{\sigma}
\end{equation}$$

y luego

$$\begin{equation}
\hat{x} =  \varphi \hat{a} + \beta
\end{equation}$$

la red debe calcular y almacenar $\mu$ y $\sigma$ (parametros no entrenables). Por otro lado, debe calcular y entrenar a $\varphi$ y $\beta$, por lo que por cada mapa de activacion almacena 4 pesos pero solo entrena 2. En total, primer batch tiene 96 x 4 = 384 pesos (192 entrenables) y el segundo 256 x 4 = 1024 (512 entrenables).

En resumen:
Pesos: 34,944 + 384 + 614,656 + 1024 = 651,008
Pesos entrenables: 34,944 + 192 + 614,656 + 512 = 650,304

Actividad 2

In [8]:
#Definicion de la tercera capa de AlexNet
modelAlexNet.add(ZeroPadding2D((1,1)))
modelAlexNet.add(Convolution2D(384, (3, 3), strides=(1,1), padding="valid"))
modelAlexNet.add(Activation(activation="relu"))
output_shape_third_layer = modelAlexNet.output_shape
print('Los mapas de activación de la capa 3 de convolción tienen tamaño: {}'.format(output_shape_third_layer))

Los mapas de activación de la capa 3 de convolción tienen tamaño: (None, 13, 13, 384)


Verificamos que tenga las mismas dimensiones

In [102]:
output_shape_third_layer

(None, 13, 13, 384)

Volvemos a revisar el resumen de la red

In [103]:
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
__________

Actividad 3

In [9]:
#Definicion de la cuarta capa de AlexNet
modelAlexNet.add(ZeroPadding2D((1,1)))
modelAlexNet.add(Convolution2D(384, (3, 3), strides=(1,1), padding="valid"))
modelAlexNet.add(Activation(activation="relu"))
output_shape_fourth_layer = modelAlexNet.output_shape
print('Los mapas de activación de la capa 4 de convolción tienen tamaño: {}'.format(output_shape_fourth_layer))

Los mapas de activación de la capa 4 de convolción tienen tamaño: (None, 13, 13, 384)


In [10]:
#Definicion de la quinta capa de AlexNet
modelAlexNet.add(ZeroPadding2D((1,1)))
modelAlexNet.add(Convolution2D(256, (3, 3), strides=(1,1), padding="valid"))
modelAlexNet.add(Activation(activation="relu"))
output_shape_fifth_layer = modelAlexNet.output_shape
print('Los mapas de activación de la capa 5 de convolción tienen tamaño: {}'.format(output_shape_fifth_layer))
modelAlexNet.add(MaxPooling2D((3,3), strides=(2,2)))

Los mapas de activación de la capa 5 de convolción tienen tamaño: (None, 13, 13, 256)


Nuevamente verificamos que las dimensiones son correctas. Al igual que la capa 3, las dimensiones de la capa 4 son (13, 13, 384). La quinta capa tiene dimensiones (13, 13, 256). Viendo las dimensiones de AlexNet, notamos que coinciden. Ahora, revisemos el resumen del modelo para ver la cantidad de parametros:

In [0]:
modelAlexNet.summary()

Podemos notar que la cuarta capa tiene más parámetros que la tercera capa, porque a pesar de tener la misma estrucutra (filtros de igual tamaño pero distinta profundidad, mapas de activación de igual tamaño), vemos que su input es distinto. La tercera recibe un input de (13, 13, 256), mientras que la cuarta recibe un input de (13, 13, 384). Por lo tanto, los filtros van a tener profundidades distintas lo que se traduce en una distinta cantidad de parametros.

Por otra parte, notamos que la quinta capa recibe el mismo input que la cuarta, pero como la quinta tiene menos mapas de activación, la cantidad de parametros que debe manejar es menor. 

Por último, notamos que como no incluimos normalización de los batches, la cantidad de parametros no entrenables no cambia.

Actividad 4

Actualmente tenemos la matriz como un tensor de 3D. Como ahora tenemos que agregar las capas densas para clasificar, debemos tener la matríz como un vector de una dimensión, para luego poder hacer correctamente las conexiones con la capa densa. Para eso, usamos el comando Flatten, que lo que hace es poner en un solo vector todos los datos del tensor 3D, por lo que tendria dimensiones 6 x 6 x 256 = 9216. Agregamos la capa:

In [0]:
modelAlexNet.add(Flatten())

In [104]:
modelAlexNet.output_shape

(None, 10)

Notamos que esto efectivamente hizo lo que necesitabamos, porque ahora el modelo tiene dimensiones (1, 9216).

Actividad 5

In [0]:
#Definicion de la capa 6 de AlexNet
modelAlexNet.add(Dense(4096, input_shape=(9216,), activation='relu', kernel_initializer='glorot_uniform', use_bias=True))
modelAlexNet.add(Dropout(0.5))
output_shape_sixth_layer = modelAlexNet.output_shape

Esta capa deberia tener dimensiones de (1, 4092), por lo que verificamos con modelAlexNet:

In [0]:
output_shape_sixth_layer

Podemos notar que tiene las dimensiones correctas. Ahora, analizemos la cantidad de parametros que lleva la red por el momento:

In [0]:
modelAlexNet.summary()

Esta ultima capa esta compuesta por 4096 neuronas y recibe como input un vector de (1, 9216). Al ser una capa Dense (fully connected), sabemos que todas las entradas se conectan con todas la neuronas de la capa, por lo que la cantidad de parametros de la capa sería 4096 * 9216 = 37,752,832. Este es un numero muy grande, son 37 millones de parametros que provinen solamente de la primera capa del clasificador. 

Esto nos hace notar lo 'eficientes' en cuanto a memoria y a entrenamiento que son las CNN. Si sumamos las 5 capas de convolucion obtenemos 3,748,608 parametros, mientras que solo esta primera capa de la FC, tiene 37,752,832 parametros (debe entrenar y almacenar aprox 10 veces más).

En total tenemos 41,501,440 parametros donde solo 704 no son entrenables. Esta cantidad no ha cambiado porque nuevamente no hemos agregado un batch normalization.

Actividad 6

In [0]:
#Definición de la capa 7 de AlexNet
modelAlexNet.add(Dense(4096, activation='relu', kernel_initializer='glorot_uniform', use_bias=True))
modelAlexNet.add(Dropout(0.5))
output_shape_seventh_layer = modelAlexNet.output_shape

In [0]:
#Definición de la capa 8 de AlexNet
modelAlexNet.add(Dense(10, activation='softmax', kernel_initializer='glorot_uniform', use_bias=True))
output_shape_eighths_layer = modelAlexNet.output_shape

In [0]:
modelAlexNet.summary()

Cantidad de filtros: 1,376
Cantidad total de parametros: 62,379,752
Cantidad de parámetros entrenables: 62,379,048
Cantidad de parámetros no entrenables: 704

Resumiendo las capas:
    1. Filtros: 96
       Cantidad de neuronas por filtro: 55 x 55 = 3,025
       Cantidad total de neuronas de capa: 290,400
       Parámetros: 34,944 (filtros) + 384 (batch normalization)
       
    2. Filtros: 256
       Cantidad de neuronas por filtro: 27 x 27 = 729
       Cantidad total de neuronas de capa: 186,624
       Parámetros: 614,656 (filtros) + 1024 (batch normalization)
       
    3. Filtros: 384
       Cantidad de neuronas por filtro: 13 x 13 = 169
       Cantidad total de neuronas de capa: 64,896
       Parámetros: 885,120 (filtros)
       
    4. Filtros: 384
       Cantidad de neuronas por filtro: 13 x 13 = 169
       Cantidad total de neuronas de capa: 64,896
       Parámetros: 1,327,488 (filtros)
       
    5. Filtros: 256
       Cantidad de neuronas por filtro: 13 x 13 = 169
       Cantidad total de neuronas de capa: 43,264
       Parámetros: 884,992 (filtros)
       
    6. Cantidad total de neuronas de capa: 4,096
       Parámetros: 37,752,832
    
    7. Cantidad total de neuronas de capa: 4,096 
       Parámetros: 16,781,312
    
    8. Cantidad total de neuronas de capa: 1,000
       Parámetros: 4,097,000
 
La capa 3 y 4 llevan 384 filtros, siendo las capas con mayor cantidad de filtros. Por otro lado, la primera capa densa es la que tiene más parámetros, 37,752,832.
 
Como mencioné anteoriormente, notamos que las capas de convolución (1 - 5) no aportan tantos parámetros a la red. Cada las neuronas que componen un filtro comparten los pesos, por lo que se reduce mucho la cantidad de pesos por capa. Esto es principalmente para que todas las neuronas de ese filtro se activen cuando encuentren el mismo patrón, es decir, cuando el producto punto entre el input y los pesos (iguales para todos) sea alto. De esta manera, cada mapa de activación es capas de detectar un patrón sobre su input. Luego, la siguiente capa de convolución va a encontrar patrones sobre estos patrones ya encontrados por cada filtro, construyendo sobre estos y encontrando objetos de mayer abstracción. En total, estas capas proveen 3,748,608 pesos.

Luego, agregamos esta otras 3 capas de fully-connected donde se conectan todas las neuronas con todas. Esto implica el entrenamiento y almacenamiento de muchos pesos. Sumando las 3 capas densas obtenemos 58,631,144 pesos. Obviamente, esta suma es muy alta por lo que requiere alto entrenamiento y varios datos para poder obtener resultados relativamente buenos. 

La arquitectura de las capas de convolución es óptima porque necesita pocos parametros y aún asī es capas de detectar figuras de alto nivel de complejidad; las cuales luego serán clasificadas como: caras, objetos, humanos etc... Las 3 capas densas de clasificación aportan muchos parámetros, por lo que es costoso entrenarla. Creo que una buena práctica sería probar con clasificadores más simples (SVM o KNN) primero. Si se obtienen buenos resultados con esos no es necesario implementar estas 3 capas, pero puede darse el caso de que la cantidad de features sea demasiado alta comparado con la cantidad de datos causando que estos clasificadores no clasifiquen bien. En ese caso necesitaríamos estas capas densas que se comportan bien independiente de la dimensionalidad de los datos.


Actividad 7

Comenzamos cargando la data del directorio provisto

In [0]:
import scipy.io as sio
import os

In [0]:
def load_mat_set(directory):
    #Load the clases
    cls = os.listdir(directory)
    #Create empty arrays
    X = []
    Y = []
    #Load the set
    for cl in cls:
        for mat in os.listdir(directory + '/' + cl):
            X.append(sio.loadmat(directory + '/' + cl + '/' + mat)['stored'][0])
            Y.append(cl)
        
    return X, Y

X_train , Y_train = load_mat_set('./MIT-10-Classes/Feats/TrainSet')
X_test, Y_test = load_mat_set('./MIT-10-Classes/Feats/TestSet')

In [0]:
#Print the dimensions of dataset
print('X_train : {}'.format(len((X_train))))
print('Y_train : {}'.format(len((Y_train))))
print('X_test : {}'.format(len((X_test))))
print('Y_test : {}'.format(len((Y_test))))

Para esta activdad vamos a entrenar un SVM. Para el SVM, vamos a usar la libreria skilearn que provee un SVM ya implementado

In [0]:
from sklearn import svm

In [0]:
#Instance a SVC model from sklearn. LinearSVC is a type of SVM for multiclass clasification.
#LinearSVC trains n_classes models using one-vs-rest clasifiers (each model clasifies one class). 
#For classifying, every classifier is tested and the result with the higher probability is chosen.
clf = svm.LinearSVC()

In [0]:
clf.fit(X_train, Y_train)

Primero, vamos a probar el modelo sobre el set de entrenamiento

In [0]:
Y_train_pred = clf.predict(X_train)
accuracy_score(Y_train, Y_train_pred)

El clasificador tiene 100% efectividad sobre el set de entrenamiento. Esto podría indicar que el modelo calló en over-fitting, como mencionamos anteriormente, esto podría ocurrir porque la dimensioón de los datos - 4096 - es mucho mayor que la cantidad de datos - 799 -. Para verificar esto, veamos el rendimiento sobre el set de test

In [0]:
Y_pred = clf.predict(X_test)

In [0]:
accuracy_score(Y_test, Y_pred)

Podemos notar que tiene una accuracy score de 97%, lo que es bastante bueno. Como se parecen ambos rendimientos, podemos descartar el hecho de que el modelo se over-fittio porque efectivamente pudo clasificar muy bien el set de test. Adicionalmente mostramos la matriz de confusión

In [0]:
confusion_matrix(Y_test, Y_pred)

Actividad 8

Comenzamos compilando el modelo creado. Vamos a usar la función de perdida categorical_crossentropy que nos permite clasificar multiples clases . Vamos a usar el optimizador adam porque hemos visto en otras CNN que funciona muy bien.

La estructura de AlexNet que haciamos creado anteriormente realmente no nos funciona para este set de datos. Nosotros habiamos creado una capa densa al final que tenia 1000 neuronas, es decir, que clasificaba 1000 categorias. El dataset MIT-10-Classes contiene solo 10 clases, por lo que debemos remover esa ultima capa y volver a agregar una densa de solo 10 neuronas:

In [0]:
modelAlexNet.layers.pop()
modelAlexNet.add(Dense(10, activation='softmax', kernel_initializer='glorot_uniform', use_bias=True))
modelAlexNet.summary()

In [0]:
modelAlexNet.compile(optimizer=optimizers.Adam(lr=5e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
#Load the images
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './MIT-10-Classes/Imgs/TrainSetImgs',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


Found 799 images belonging to 10 classes.
Found 201 images belonging to 10 classes.


In [17]:
modelAlexNet.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples / 32,
        epochs=25,
        verbose=1)

Epoch 1/50
25/24 [==============================] - 9s 340ms/step - loss: 3.0145 - acc: 0.1100
Epoch 2/50
25/24 [==============================] - 7s 275ms/step - loss: 2.1548 - acc: 0.2179
Epoch 3/50
25/24 [==============================] - 7s 274ms/step - loss: 2.0155 - acc: 0.2792
Epoch 4/50
25/24 [==============================] - 7s 272ms/step - loss: 1.9025 - acc: 0.3328
Epoch 5/50
25/24 [==============================] - 7s 275ms/step - loss: 1.7763 - acc: 0.3729
Epoch 6/50
25/24 [==============================] - 7s 274ms/step - loss: 1.6151 - acc: 0.4105
Epoch 7/50
25/24 [==============================] - 7s 274ms/step - loss: 1.5132 - acc: 0.4705
Epoch 8/50
25/24 [==============================] - 7s 274ms/step - loss: 1.4552 - acc: 0.5005
Epoch 9/50
14/24 [===============>..............] - ETA: 3s - loss: 1.2674 - acc: 0.5861

25/24 [==============================] - 7s 273ms/step - loss: 1.2696 - acc: 0.5795
Epoch 10/50
25/24 [==============================] - 7s 273ms/step - loss: 1.1245 - acc: 0.6144
Epoch 11/50
25/24 [==============================] - 6s 255ms/step - loss: 0.8619 - acc: 0.6971
Epoch 12/50
25/24 [==============================] - 7s 277ms/step - loss: 0.7394 - acc: 0.7648
Epoch 13/50
25/24 [==============================] - 7s 273ms/step - loss: 0.7013 - acc: 0.7748
Epoch 14/50
25/24 [==============================] - 7s 273ms/step - loss: 0.5136 - acc: 0.8223
Epoch 15/50
25/24 [==============================] - 7s 271ms/step - loss: 0.4315 - acc: 0.8623
Epoch 16/50
25/24 [==============================] - 7s 272ms/step - loss: 0.4027 - acc: 0.8610
Epoch 17/50
16/24 [==================>...........] - ETA: 2s - loss: 0.3003 - acc: 0.9160

25/24 [==============================] - 7s 273ms/step - loss: 0.2992 - acc: 0.9098
Epoch 18/50
25/24 [==============================] - 7s 274ms/step - loss: 0.2304 - acc: 0.9249
Epoch 19/50
25/24 [==============================] - 7s 273ms/step - loss: 0.2728 - acc: 0.9134
Epoch 20/50
25/24 [==============================] - 7s 272ms/step - loss: 0.4047 - acc: 0.8922
Epoch 21/50
25/24 [==============================] - 7s 274ms/step - loss: 0.3157 - acc: 0.9012
Epoch 22/50
25/24 [==============================] - 7s 273ms/step - loss: 0.2021 - acc: 0.9274
Epoch 23/50
25/24 [==============================] - 7s 274ms/step - loss: 0.1527 - acc: 0.9499
Epoch 24/50
25/24 [==============================] - 7s 274ms/step - loss: 0.1705 - acc: 0.9475
Epoch 25/50
17/24 [===================>..........] - ETA: 2s - loss: 0.2273 - acc: 0.9337

25/24 [==============================] - 7s 276ms/step - loss: 0.2240 - acc: 0.9312
Epoch 26/50
25/24 [==============================] - 7s 274ms/step - loss: 0.2176 - acc: 0.9448
Epoch 27/50
25/24 [==============================] - 7s 273ms/step - loss: 0.2533 - acc: 0.9337
Epoch 28/50
25/24 [==============================] - 7s 271ms/step - loss: 0.1786 - acc: 0.9474
Epoch 29/50
25/24 [==============================] - 7s 276ms/step - loss: 0.1251 - acc: 0.9637
Epoch 30/50
25/24 [==============================] - 7s 272ms/step - loss: 0.1669 - acc: 0.9500
Epoch 31/50
25/24 [==============================] - 7s 272ms/step - loss: 0.1519 - acc: 0.9500
Epoch 32/50
25/24 [==============================] - 7s 273ms/step - loss: 0.0859 - acc: 0.9737
Epoch 33/50
16/24 [==================>...........] - ETA: 2s - loss: 0.0783 - acc: 0.9746

25/24 [==============================] - 7s 273ms/step - loss: 0.0705 - acc: 0.9787
Epoch 34/50
25/24 [==============================] - 7s 274ms/step - loss: 0.1080 - acc: 0.9687
Epoch 35/50
25/24 [==============================] - 7s 275ms/step - loss: 0.2308 - acc: 0.9450
Epoch 36/50
25/24 [==============================] - 7s 272ms/step - loss: 0.2349 - acc: 0.9412
Epoch 37/50
25/24 [==============================] - 7s 271ms/step - loss: 0.1962 - acc: 0.9412
Epoch 38/50
25/24 [==============================] - 7s 273ms/step - loss: 0.2061 - acc: 0.9448
Epoch 39/50
25/24 [==============================] - 7s 268ms/step - loss: 0.1793 - acc: 0.9487
Epoch 40/50
25/24 [==============================] - 7s 270ms/step - loss: 0.0557 - acc: 0.9812
Epoch 41/50
16/24 [==================>...........] - ETA: 2s - loss: 0.1019 - acc: 0.9726

25/24 [==============================] - 7s 272ms/step - loss: 0.0999 - acc: 0.9700
Epoch 42/50
25/24 [==============================] - 7s 271ms/step - loss: 0.1116 - acc: 0.9700
Epoch 43/50
25/24 [==============================] - 7s 271ms/step - loss: 0.1471 - acc: 0.9687
Epoch 44/50
25/24 [==============================] - 7s 276ms/step - loss: 0.2206 - acc: 0.9411
Epoch 45/50
25/24 [==============================] - 7s 273ms/step - loss: 0.2958 - acc: 0.9387
Epoch 46/50
25/24 [==============================] - 7s 280ms/step - loss: 0.1771 - acc: 0.9486
Epoch 47/50
25/24 [==============================] - 7s 278ms/step - loss: 0.1990 - acc: 0.9575
Epoch 48/50
25/24 [==============================] - 7s 278ms/step - loss: 0.1885 - acc: 0.9625
Epoch 49/50
17/24 [===================>..........] - ETA: 2s - loss: 0.1912 - acc: 0.9593

25/24 [==============================] - 7s 277ms/step - loss: 0.1554 - acc: 0.9636
Epoch 50/50
25/24 [==============================] - 7s 280ms/step - loss: 0.1021 - acc: 0.9661


In [18]:
print("Saving Model")
modelAlexNet.save('modelAlexNet.h5')

Saving Model


In [0]:
import scipy.io as sio
import numpy as np
import cv2
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [0]:
def load_test_set(directory):
    #Load the clases
    cls = os.listdir(directory)
    #Create empty arrays
    X = []
    Y = []
    #Load the set
    for cl in cls:
        for img in os.listdir(directory + '/' + cl):
            img_arr = img_to_array(load_img(directory + '/' + cl + '/' + img))
            resized = cv2.resize(img_arr, (224, 224))
            X.append(resized)
            Y.append(cl)
    return np.array(X), np.array(Y)

X_test, Y_test_string = load_test_set('./MIT-10-Classes/Imgs/TestSetImgs')

In [0]:
# Convert string classes to vector of dim (1,10). Vector has a 1 on the
# corresponding class and 0 on the rest.
encoder = LabelEncoder()
encoder.fit(Y_test_string)
encoded_Y = encoder.transform(Y_test_string)
Y_test = to_categorical(encoded_Y)

In [98]:
Y_pred = modelAlexNet.predict(X_test)
res = 0
for i in range(Y_pred.shape[0]):
  if np.array_equal(Y_pred[i], Y_test[i]) == True:
    res += 1
print(res/Y_pred.shape[0])

0.19900497512437812
